#12_선형분류_다중분류

### ML/DL 라이브러리 불러오기

In [1]:
import torch
import torch.optim as optim
# random Seed 고정하기
torch.manual_seed(1)

##1. 데이터셋불러오기

In [2]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

In [3]:
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [4]:
print(x_train[:5])
print(y_train[:5])

tensor([[1., 2., 1., 1.],
        [2., 1., 3., 2.],
        [3., 1., 3., 4.],
        [4., 1., 5., 5.],
        [1., 7., 5., 5.]])
tensor([2, 2, 2, 1, 1])


정답 데이터

In [5]:
print(y_train)
print(y_train.unsqueeze(1))

tensor([2, 2, 2, 1, 1, 1, 0, 0])
tensor([[2],
        [2],
        [2],
        [1],
        [1],
        [1],
        [0],
        [0]])


One-hot Encoding: 정답 데이터

In [6]:
nb_class = 3
nb_data = len(y_train)
y_one_hot = torch.zeros(nb_data, nb_class)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

print(y_one_hot)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


##5. 모델 학습: Softmax

In [7]:
# Softmax, Cross-entropy
import torch.nn.functional as F

In [8]:
nb_class = 3
nb_data = len(y_train)

###1. Full Code with Softmax & One-hot Encoding

In [9]:
# Initialization: W and b
W = torch.zeros((4, nb_class), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# Opimizer
optimizer = optim.SGD([W, b], lr = 0.01)

# 파라미커 100번 갱신 반복하기
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # Softmax Hypothesis
  hypothesis = F.softmax(x_train.matmul(W)+b, dim = 1)

  # One-hot Encoding
  y_one_hot = torch.zeros(nb_data, nb_class)
  y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

  # Cost
  cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim = 1))).sum(dim = 1).mean()

  # Cost로 파라미터 W, b 갱신
  optimizer.zero_grad() # Optimizer의 gradient 0으로 초기화
  cost.backward() # Cost로 Optimizer 알고리즘 수행
  optimizer.step() # W, b 갱신

  #100번마다 갱신 상황 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 1.036919
Epoch  200/1000 Cost: 1.010905
Epoch  300/1000 Cost: 0.991541
Epoch  400/1000 Cost: 0.974942
Epoch  500/1000 Cost: 0.959928
Epoch  600/1000 Cost: 0.946149
Epoch  700/1000 Cost: 0.933492
Epoch  800/1000 Cost: 0.921898
Epoch  900/1000 Cost: 0.911302
Epoch 1000/1000 Cost: 0.901625


#### 2. Full Code with Softmax & Cross-Entropy

In [10]:
# Initialization: W and b
W = torch.zeros((4, nb_class), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# Opimizer
optimizer = optim.SGD([W, b], lr = 0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # Softmax Hypothesis
  hypothesis = F.softmax(x_train.matmul(W)+b, dim = 1)

  # Cost with Cross-entropy: One-hot Encoding 필요 없다
  cost = F.cross_entropy((x_train.matmul(W) + b), y_train)

  # Cost로 파라미터 W, b 갱신
  optimizer.zero_grad() # Optimizer의 gradient 0으로 초기화
  cost.backward() # Cost로 Optimizer 알고리즘 수행
  optimizer.step() # W, b 갱신

  #100번마다 갱신 상황 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.850816
Epoch  200/1000 Cost: 0.784908
Epoch  300/1000 Cost: 0.744590
Epoch  400/1000 Cost: 0.714646
Epoch  500/1000 Cost: 0.690688
Epoch  600/1000 Cost: 0.670780
Epoch  700/1000 Cost: 0.653828
Epoch  800/1000 Cost: 0.639125
Epoch  900/1000 Cost: 0.626180
Epoch 1000/1000 Cost: 0.614640


## 6. 결과 분석

Softmax Hypothesis

In [11]:
hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)

print(hypothesis)

tensor([[0.1417, 0.2281, 0.6302],
        [0.0747, 0.3657, 0.5596],
        [0.0181, 0.4477, 0.5343],
        [0.0088, 0.4927, 0.4986],
        [0.6136, 0.2927, 0.0937],
        [0.2573, 0.7295, 0.0132],
        [0.5776, 0.3942, 0.0282],
        [0.6400, 0.3469, 0.0130]], grad_fn=<SoftmaxBackward>)


argmax를 이용한 예측

In [12]:
# argmax: 가장 큰 값의 인덱스 반환
predict = torch.argmax(hypothesis, dim = 1)

print(predict)

tensor([2, 2, 2, 2, 0, 1, 0, 0])


정답

In [13]:
print(y_train)

tensor([2, 2, 2, 1, 1, 1, 0, 0])


채점

In [14]:
correct_prediction = predict.float() == y_train

print(correct_prediction)

tensor([ True,  True,  True, False, False,  True,  True,  True])


정확도

In [15]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)

print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 75.00% for the training set.
